## 使用Func运维Storm集群

1.检查集群中所有机器的运行状态
```bash
$ func "*" ping
[ ok ... ] storm010099204068.cm8
[ ok ... ] storm010099204069.cm8
[ ok ... ] storm010099204070.cm8
[ FAILED ] storm010207000061.cm3
...
```
更准确地说，上面的命令是检查funcd的运行状态，但因为funcd注册为开机自动运行服务，所以funcd服务不存在，很可能机器挂了。


2.文件部署
```
$ func "*.cm8" copyfile --file=/tmp/foo --remotepath=/tmp/foo
```
将overload上/tmp/foo部署到与"*.cm8"匹配的所有机器，避免了重复登录目标主机

3.修改配置文件

这里举一个比较实用的例子：各个kafka server需要拥有不同的brokerid，因此需要在部署过程中一一修改配置文件，这里用一个简单的脚本一步搞定。
```bash
#!/bin/sh

i=500

while read minion
do
    sudo func "${minion}" call command run "sed -i -e '20 c brokerid='$((++i))'' /home/admin/saf/etc/kafka.conf"
done
```

4.进程管理

```
$ func "storm010207000085.cm3" call command run "supervisorctl status kafka"
(
    'storm010207000085.cm3',
    [0, 'supervisor                       RUNNING   pid 30416, uptime 3:32:10\n', '']
)

$ func "storm010207000085.cm3" call command run "supervisorctl stop kafka"
('storm010207000085.cm3', [0, 'kafka: stopped\n', ''])

$ func "storm010207000085.cm3" call command run "supervisorctl start kafka"
('storm010207000085.cm3', [0, 'kafka: started\n', ''])
```